# Nettoyage des données scrappées sur Wikipédia 

Dans cette partie, nous procédons au nettoyage des données que nous avons scrappé sur Wikipédia (vous pouvez vous réferer au dossier wiki_scrap et le fichier fonction_scrapping pour voir la première étape). 

En effet, nous avons récupéré en plusieurs fichiers csv les données brutes de pages de Wiki qui rescencent les nominations, catégories et les victoires dans les compétitions et festivals de cinéma tels que Oscar (Academy awards), César, Festival de Cannes et Festival de Venise. 

## Objectifs de la partie nettoyage 

* D'abord, nettoyer les bases une à une, renommer les colonnes, éviter les répitions et supprimer les lignes qui sont inutiles (comme nous avons scrappé tous les tableaux sur les pages étant donné que les tables ne suivent pas toujours le même schema)
* Créer une seule base à partir des tables nettoyées - et différencier deux types de compétitions - festivals et compétitions plus mainstream. 
Nous voulons récupérer ue base de données unique à la fin qui aura la forme suivante: {Name, Year, Dir_or_prod, Studio, Award_mains, Awards_mains_cat, Award_fest, Win} 
    * La colonne Name sera le nom du film
    * La colonne Year est l'année de sortie
    * La colonne Dir_or_prod est le réalisateur ou le producteur du film
    * La colonne Studio est le studio de production (si mentionné)
    * La colonne Award_mains est la variable qui enregistre si le film a été nominé ou a gagné à la compétition plus "mainstream", plus commune telle que Academy Awards ou le prix César. La variable prendra la valeur 0 si le film n'a pas été nominé ni a gagné, 1 s'il a été nominé et 2 s'il a gagné.
    * Award_mains_cat enregistre la catégorie dans laquelle le film a été nominé/qu'il a remporté
    * De même que pour Award_mains, Award_fest enregistre si le film a été nominé ou a gagné dans la compétition d'un des grands festivals - Cannes et Venise.
    


- [ ] ac_bf - séparer film studio et producteur (galère) 
- [ ] ac_bd - OK
- [ ] ac_bc OK
- [ ] can_gp OK
- [ ] can_jr OK
- [ ] can_po OK
- [ ] ces_bd OK
- [ ] ces_bf OK
- [ ] ven_lor OK
- [ ] ven_larg OK
- [ ] ven_gj OK

In [37]:
import pandas as pd
import re

In [2]:
base = pd.read_csv('../API/MoviesPopular.csv')
print(base.columns)

Index(['Unnamed: 0', 'adult', 'backdrop_path', 'belongs_to_collection',
       'budget', 'genres', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count'],
      dtype='object')


In [177]:
#Chargement des bases scrappées sur wikipédia pour nettoyage et fusion avec la base de données récupérée sur TMDB

#Academy award for best cinematography 
ac_bc = pd.read_csv('../wiki_scrap/academy_bc.csv')
# academy award for best film 
ac_bf = pd.read_csv('../wiki_scrap/academy_bf.csv')
#Academy award for best director 
ac_bd = pd.read_csv('../wiki_scrap/academy_bd.csv')
#Cannes Grand Prix
can_gp = pd.read_csv('../wiki_scrap/cannes_gp.csv')
#Cannes Jury prize
can_jr = pd.read_csv('../wiki_scrap/cannes_jury.csv')
#Cannes Palme d'or
can_po = pd.read_csv('../wiki_scrap/palme_or.csv')
#César Best director 
ces_bd = pd.read_csv('../wiki_scrap/cesar_bd.csv')
#Cesar Best Film 
ces_bf = pd.read_csv('../wiki_scrap/cesar_bf.csv')
#Venice Lion d'or
ven_lor = pd.read_csv('../wiki_scrap/lion_or.csv')
#Venice Lion d'argent 
ven_larg = pd.read_csv('../wiki_scrap/lion_argent.csv')
#Venice Grand Jury Prix
ven_gj = pd.read_csv('../wiki_scrap/venice_gj.csv')


In [4]:
#Pour vérifier si le chargement est bien réussi 

print(ac_bc.columns)
print(ac_bd.columns) 
print(ac_bf.columns) 

print(ces_bd.columns) 
print(ces_bf.columns) 

print(can_gp.columns)
print(can_jr.columns) 
print(can_po.columns)

print(ven_lor.columns) 
print(ven_larg.columns) 
print(ven_gj.columns)

Index(['Category', 'Name', 'Superlative', 'Year', 'Notes', 'Film', 'Nominees',
       'Awards', 'Nominations', 'Recipient'],
      dtype='object')
Index(['0', 'Year', 'Director(s)', 'Film', 'Ref.', 'Wins', 'Director',
       'Nominations', 'Record', 'Age'],
      dtype='object')
Index(['Year of Film Release', 'Film', 'Film Studio',
       'Film Studio/Producer(s)', 'Producer(s)', 'Production Company',
       'Nominations', 'Wins'],
      dtype='object')
Index(['Superlative', 'Men', 'Men.1', 'Women', 'Women.1', 'Year',
       'Winner and nominees', 'English title', 'Original title',
       'Winners and nominees', 'Wins', 'Director', 'Nominations'],
      dtype='object')
Index(['Year', 'English title', 'Original title', 'Director'], dtype='object')
Index(['('Year', '1960s', 'Awarded as "Grand Prix Spécial du Jury"')',
       '('English Title', '1960s', 'Awarded as "Grand Prix Spécial du Jury"')',
       '('Original Title', '1960s', 'Awarded as "Grand Prix Spécial du Jury"')',
       '('D

In [108]:
#Création de la fonction qui convertit la colonne Year en format date 

def conv_date(df): 
    df['Year'] = df['Year'].astype(str)
    df['Year'] = df['Year'].str.extract('(\d{4})', expand=False)
    df['Year'] = pd.to_datetime(df['Year'], format='%Y', errors='coerce')
    return df

In [109]:
#Nettoyage et mise en conformité de la base de données Academy 

#Best-cinematography 

#Supprimer les 3 premières et 3 dernières colonnes et 13 premières lignes qui référent à un tableau qui ne nous intéresse pas dans notre code
ac_bc_cl = ac_bc.drop(['Category', 'Name', 'Superlative', 'Notes', 'Awards', 'Nominations', 'Recipient'], axis=1)
ac_bc_cl = ac_bc_cl.iloc[13:, :]
#conversion en type date de la colonne 'Year'

ac_bc_cl = conv_date(ac_bc_cl)
ac_bc_cl = ac_bc_cl.dropna(subset=['Film', 'Nominees'], how='all')
ac_bc_cl

,Year,Film,Nominees
13,1927-01-01,Sunrise: A Song of Two Humans,Charles Rosher Karl Struss
14,1927-01-01,The Devil Dancer,George Barnes
15,1927-01-01,The Magic Flame,George Barnes
16,1927-01-01,Sadie Thompson,George Barnes
18,1928-01-01,White Shadows in the South Seas,Clyde De Vinna
...,...,...,...
782,2022-01-01,All Quiet on the Western Front,James Friend
783,2022-01-01,"Bardo, False Chronicle of a Handful of Truths",Darius Khondji
784,2022-01-01,Elvis,Mandy Walker
785,2022-01-01,Empire of Light,Roger Deakins


In [249]:
#Meilleur réalisateur - Academy 

ac_bd_cl = ac_bd.drop (['0', 'Ref.', 'Wins', 'Director', 'Nominations', 'Record', 'Age' ], axis=1)
ac_bd_cl = ac_bd_cl.iloc[:473, :]
#conversion de l'année en type date 
ac_bd_cl = conv_date(ac_bd_cl)
#renommer la colonne director
ac_bd_cl.rename(columns={'Film':'English_title','Director(s)': 'Director'}, inplace=True)
#suppression des lignes inutiles (celles qui ne contiennent que l'année sans film
ac_bd_cl = ac_bd_cl.dropna(subset=['Director', 'English_title'], how='all')

#supprimer le nom de studio dans la colonne Director
ac_bd_cl['Director'] = ac_bd_cl['Director'].str.split('(', n=1).str[0]

#reset index 
ac_bd_cl = ac_bd_cl.reset_index(drop=True)

ac_bd_cl



,Year,Director,English_title
0,1927-01-01,Frank Borzage,7th Heaven
1,1927-01-01,Herbert Brenon,Sorrell and Son
2,1927-01-01,King Vidor,The Crowd
3,1927-01-01,Lewis Milestone,Two Arabian Knights
4,1927-01-01,Ted Wilde,Speedy
...,...,...,...
467,2022-01-01,Daniel Kwan and Daniel Scheinert,Everything Everywhere All at Once
468,2022-01-01,Todd Field,Tár
469,2022-01-01,Martin McDonagh,The Banshees of Inisherin
470,2022-01-01,Ruben Östlund,Triangle of Sadness


In [250]:
#Academy - Best Film 
ac_bf_cl=ac_bf


ac_bf_cl.rename(columns={'Year of Film Release': 'Year'}, inplace=True)

ac_bf_cl = conv_date(ac_bf_cl)

ac_bf_cl = ac_bf_cl.dropna(subset=['Film'], how='all')
#Problème dans la base - les producteurs et les studios de film ne sont pas séparés du tout, et mélangés au sein de 3 colonnes 

# Extraction des noms de producteurs de la colonne Film Studio (les producteurs sont souvent mentionnés entre parenthèses)
ac_bf_cl['Film_producers'] = ac_bf_cl['Film Studio'].str.extract(r'\(([^)]+)\)')
#supprimer la mention producer ou producers 
ac_bf_cl['Film_producers'] = ac_bf_cl['Film_producers'].str.split(', producer', n=1).str[0]
#lier avec la colonne producers 
ac_bf_cl['Film_producers'] = ac_bf_cl['Film_producers'].combine_first(ac_bf_cl['Producer(s)'])
ac_bf_cl = ac_bf_cl.drop(['Producer(s)', 'Wins', 'Nominations', 'Production Company'], axis=1)

#suppression du nom de pans la colonne film studio
ac_bf_cl['Film Studio'] = ac_bf_cl['Film Studio'].str.split('(', n=1).str[0]

#Maintenant, séparons la colonne Film Studio/Producers 
ac_bf_cl['Film Studio/Producer(s)'] = ac_bf_cl['Film Studio/Producer(s)'].str.split('(', n=1).str[0]
#D'abord récupérons les noms de studios présents (manuellement car entrées peu nombreuses) 
studio = ['Metro-Goldwyn-Mayer', 'Columbia', '20th Century-Fox', '	Paramount']
#Recréer une colonne à base des noms de studio 
ac_bf_cl['Studio'] = ac_bf_cl['Film Studio/Producer(s)'].where(ac_bf_cl['Film Studio/Producer(s)'].isin(studio))
#La lier avec Film Studio d'origine 
ac_bf_cl['Film Studio'] = ac_bf_cl['Film Studio'].combine_first(ac_bf_cl['Studio'])
#Supprimer les entrées dans la colonne 
ac_bf_cl = ac_bf_cl [~ac_bf_cl['Film Studio/Producer(s)'].isin(studio)]
#Combiner avec la liste des producteurs 
ac_bf_cl['Film_producers'] = ac_bf_cl['Film_producers'].combine_first(ac_bf_cl['Film Studio/Producer(s)'])

#renommer les colonnes 
ac_bf_cl.rename(columns={'Film Studio': 'Film_Studio', 'Film':'English_title'}, inplace=True)
#supprimer la référence non nécessaire
ac_bf_cl['English_title'] = ac_bf_cl['English_title'].str.split('[', n=1).str[0]
ac_bf_cl = ac_bf_cl.drop(['Film Studio/Producer(s)', 'Studio'], axis=1)
ac_bf_cl = ac_bf_cl.dropna(subset=['English_title', 'Year'], how='all')

#reset index 
ac_bf_cl = ac_bf_cl.reset_index(drop=True)
ac_bf_cl

/tmp/ipykernel_4667/2902350124.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ac_bf_cl['Film_producers'] = ac_bf_cl['Film Studio'].str.extract(r'\(([^)]+)\)')
/tmp/ipykernel_4667/2902350124.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ac_bf_cl['Film_producers'] = ac_bf_cl['Film_producers'].str.split(', producer', n=1).str[0]
/tmp/ipykernel_4667/2902350124.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

,Year,English_title,Film_Studio,Film_producers
0,1927-01-01,Wings,Famous Players–Lasky,"Lucien Hubbard, Jesse L. Lasky, B.P. Schulberg..."
1,1927-01-01,7th Heaven,Fox,William Fox
2,1927-01-01,The Racket,The Caddo Company,Howard Hughes
3,1928-01-01,The Broadway Melody,Metro-Goldwyn-Mayer,Irving Thalberg & Lawrence Weingarten
4,1928-01-01,Alibi,Feature Productions,Roland West
...,...,...,...,...
583,2022-01-01,The Fabelmans,NaN,"Kristie Macosko Krieger, Steven Spielberg, Ton..."
584,2022-01-01,Tár,NaN,"Todd Field, Alexandra Milchan, and Scott Lambert"
585,2022-01-01,Top Gun: Maverick,NaN,"Tom Cruise, Christopher McQuarrie, David Ellis..."
586,2022-01-01,Triangle of Sadness,NaN,Erik Hemmendorff and Philippe Bober


In [114]:
#création d'une fonction qui permet de renommer les colonnes en une seule commande
#étant donné que les tableaux des 3 pages sur les prix à Canne susivent la même structure
def renom_cannes(df) : 
    h = df.columns
    #renommer les colonnes qui ont été mal scrappées
    df.rename(columns={h[0]: 'Year', h[1] : 'English_title' , h[2]:'Original_title', h[3]:'Director', h[4]:'Production_country' }, inplace=True)
    return df 

In [115]:
#Grand Prix - Cannes

can_gp_cl = can_gp
can_gp_cl = renom_cannes(can_gp_cl)
can_gp_cl = conv_date(can_gp_cl)
#suppression lignes non voulues avec la décennie comme valeur dans chaque colonne (scrapping du tableau difficile)
decades_to_delete = [f"{decade}s" for decade in range(1960, 2030, 10)]
can_gp_cl = can_gp_cl.loc[~can_gp_cl['English_title'].isin(decades_to_delete)]
can_gp_cl = can_gp_cl.reset_index(drop=True)


In [116]:

can_gp_cl= can_gp_cl.iloc[:65, :5]

can_gp_cl

,Year,English_title,Original_title,Director,Production_country
0,1967-01-01,Accident,Accident,Joseph Losey,United Kingdom
1,1967-01-01,I Even Met Happy Gypsies,Skupljači perja,Aleksandar Petrović,Yugoslavia
2,1969-01-01,Ådalen 31,Ådalen 31,Bo Widerberg,Sweden
3,1970-01-01,Investigation of a Citizen Above Suspicion,Indagine su un cittadino al di sopra di ogni s...,Elio Petri,Italy
4,1971-01-01,Johnny Got His Gun,Johnny Got His Gun,Dalton Trumbo,United States
...,...,...,...,...,...
60,2019-01-01,Atlantics,Atlantique,Mati Diop,"France, Senegal, Belgium"
61,2021-01-01,Compartment No. 6,Hytti nro 6,Juho Kuosmanen,"Finland, Germany, Estonia, Russia"
62,2021-01-01,A Hero,قهرمان,Asghar Farhadi,"Iran, France"
63,2022-01-01,Close,Close,Lukas Dhont,"Belgium, Netherlands, France"


In [138]:
#Prix du Jury - Cannes

can_jr_cl = can_jr
can_jr_cl = renom_cannes(can_jr_cl)

#suppression lignes non voulues (scrapping du tableau difficile)
decades_to_delete = ["1950s"] + decades_to_delete
can_jr_cl = can_jr_cl.loc[~can_jr_cl['English_title'].isin(decades_to_delete)]
delete = ['Awarded as "Special Jury Prize"', 'Awarded as "Jury Prize"']
can_jr_cl = can_jr_cl [~can_jr_cl ['Year'].isin(delete)]

#conversion de la date et sélection des colonnes d'intérêt 
can_jr_cl = conv_date(can_jr_cl)
can_jr_cl= can_jr_cl.iloc[:81, :5]
can_jr_cl = can_jr_cl.reset_index(drop=True)

can_jr_cl


,Year,English_title,Original_title,Director,Production_country
0,1946-01-01,The Battle of the Rails,La Bataille du rail,René Clément,France
1,1951-01-01,All About Eve,All About Eve,Joseph L. Mankiewicz,United States
2,1952-01-01,We Are All Murderers,Nous sommes tous des assassins,André Cayatte,France
3,1954-01-01,Knave of Hearts,Monsieur Ripois,René Clément,France
4,1955-01-01,Lost Continent,Continente perduto,"Enrico Gras, Giorgio Moser & Leonardo Bonzi",Italy
...,...,...,...,...,...
76,2021-01-01,Ahed's Knee,הַבֶּרֶךְ,Nadav Lapid,"France, Germany, Israel"
77,2021-01-01,Memoria,Memoria,Apichatpong Weerasethakul,"Colombia, Thailand, United Kingdom, Mexico, Fr..."
78,2022-01-01,The Eight Mountains,Le otto montagne,Felix van Groeningen & Charlotte Vandermeersch,"Italy, Belgium, France"
79,2022-01-01,EO,EO,Jerzy Skolimowski,"Poland, Italy"


In [137]:
#Palme d'Or

can_po_cl = can_po 
can_po_cl = renom_cannes(can_po_cl) 
#gérer à la fois les guillemets et les apostrophes
c = 'Awarded as' + ' "Palme d' +"'" +'Or"'
supp=['Awarded as "Grand Prix"', c, 'Awarded as "Grand Prix du Festival International du Film"', 'Festival cancelled due to the COVID-19 pandemic. 56 films were announced as official selections by the festival, but no awards were presented.' ]
can_po_cl = can_po_cl [~can_po_cl ['Year'].isin(supp)]
#sélection des colonnes d'intérêt 
can_po_cl= can_po_cl.iloc[1:, :5]
can_po_cl = can_po_cl.dropna(subset=['Original_title', 'Director'], how='all')
# suppression du symbole § présent dans les cetaines entrées sur wiki 
can_po_cl['English_title'] = can_po_cl['English_title'].astype(str).apply(lambda x: x.rstrip('§'))
can_po_cl['Original_title'] = can_po_cl['Original_title'].astype(str).apply(lambda x: x.rstrip('§'))
#can_po_cl = can_po_cl[pd.notna(can_po_cl['English_title']) & ~can_po_cl['English_title'].str.endswith('§')]
can_po_cl = can_po_cl.reset_index(drop=True)


can_po_cl

,Year,English_title,Original_title,Director,Production_country
0,1946,Brief Encounter,Brief Encounter,David Lean,United Kingdom
1,1946,The Last Chance,Die Letzte Chance,Leopold Lindtberg,Switzerland
2,1946,The Lost Weekend,The Lost Weekend,Billy Wilder,United States
3,1946,María Candelaria,María Candelaria,Emilio Fernández,Mexico
4,1946,Men Without Wings,Muži bez křídel,František Čáp,Czechoslovakia
...,...,...,...,...,...
98,2019,Parasite,기생충,Bong Joon-ho,South Korea
99,2020,Festival cancelled due to the COVID-19 pandemi...,Festival cancelled due to the COVID-19 pandemi...,Festival cancelled due to the COVID-19 pandemi...,Festival cancelled due to the COVID-19 pandemi...
100,2021,Titane,Titane,Julia Ducournau,"France, Belgium"
101,2022,Triangle of Sadness,Triangle of Sadness,Ruben Östlund,Sweden


In [149]:
#César - Meilleur Réalisateur 

ces_bd_cl = ces_bd 
ces_bd_cl = ces_bd_cl.iloc[8:, 5:10]
ces_bd_cl = conv_date(ces_bd_cl)

#Combiner les deux colonnes avec les mêmes valeurs en une seule 
#certaines lignes de la première contiennent NAN où la deuxième a des valeurs

ces_bd_cl['Winners_and_nominees'] = ces_bd_cl['Winner and nominees'].combine_first(ces_bd_cl['Winners and nominees'])

# Supprimer les colonnes originales 
ces_bd_cl = ces_bd_cl.drop(['Winner and nominees', 'Winners and nominees'], axis=1)
col=ces_bd_cl.columns
ces_bd_cl.rename(columns={ col[1] : 'English_title' , col[2]:'Original_title' }, inplace=True)
ces_bd_cl = ces_bd_cl.dropna(subset=['Original_title', 'Winners_and_nominees'], how='all')
# Réinitialiser l'index
ces_bd_cl = ces_bd_cl.reset_index(drop=True)
ces_bd_cl

,Year,English_title,Original_title,Winners_and_nominees
0,1976-01-01,The Old Gun,Le Vieux Fusil,Robert Enrico
1,1976-01-01,Call me Savage,Le Sauvage,Jean-Paul Rappeneau
2,1976-01-01,The Story of Adele H.,L'Histoire d'Adèle H.,François Truffaut
3,1977-01-01,Mr. Klein,Monsieur Klein,Joseph Losey
4,1977-01-01,The Best Way to Walk,La Meilleure Façon de marcher,Claude Miller
...,...,...,...,...
248,2023-01-01,The Night of the 12th,La Nuit du 12,Dominik Moll
249,2023-01-01,Rise,En corps,Cedric Klapisch
250,2023-01-01,The Innocent,L'Innocent,Louis Garrel
251,2023-01-01,November,Novembre,Cédric Jimenez


In [156]:
ces_bf_cl = ces_bf
ces_bf_cl = conv_date(ces_bf_cl) # convertir l'année en format date
col = ces_bf_cl.columns
ces_bf_cl.rename(columns={ col[1] : 'English_title' , col[2]:'Original_title' }, inplace=True) #renommer les colonnes
ces_bf_cl = ces_bf_cl.dropna(subset=['Original_title', 'Director'], how='all') #supprimer les lignes avec NaN 

# Réinitialiser l'index
ces_bd_cl = ces_bd_cl.reset_index(drop=True)
ces_bf_cl

,Year,English_title,Original_title,Director
0,1976-01-01,The Old Gun,Le Vieux Fusil,Robert Enrico
1,1976-01-01,"Cousin, Cousine","Cousin, cousine",Jean Charles Tacchella
2,1976-01-01,Let Joy Reign Supreme,Que la fête commence,Bertrand Tavernier
3,1976-01-01,Seven Deaths by Prescription,Sept morts sur ordonnance,Jacques Rouffio
4,1977-01-01,Mr. Klein,Monsieur Klein,Joseph Losey
...,...,...,...,...
260,2023-01-01,The Night of the 12th,La Nuit du 12,Dominik Moll
261,2023-01-01,Forever Young,Les Amandiers,Valeria Bruni Tedeschi
262,2023-01-01,Rise,En corps,Cédric Klapisch
263,2023-01-01,The Innocent,L'Innocent,Louis Garrel


In [198]:
ven_lor_cl = ven_lor 
ven_lor_cl = renom_cannes(ven_lor_cl)
ven_lor_cl = ven_lor_cl.iloc[ ::, :5]

#supprimer les lignes parasites
suppr=['1946: Awarded as "Best Feature Film"', '1947–1948: Awarded as "Grand International Prize of Venice"', '1949–1953: Awarded as "Golden Lion of Saint Mark"', '1954–present: Awarded as "Golden Lion"' ] + decades_to_delete + ['1940s']
ven_lor_cl = ven_lor_cl [~ven_lor_cl ['English_title'].isin(suppr)]
#supprimer les Nan 
ven_lor_cl = ven_lor_cl.dropna(subset=['English_title'], how='all')
#supprimer les lignes où la récompense n'a pas été donnée 
ven_lor_cl = ven_lor_cl[~ven_lor_cl['English_title'].str.startswith('No award given')]
ven_lor_cl['English_title'] = ven_lor_cl['English_title'].astype(str).apply(lambda x: x.rstrip('§'))
#convertir en date 
ven_lor_cl = conv_date(ven_lor_cl)
ven_lor_cl = ven_lor_cl.reset_index(drop=True)
ven_lor_cl

,Year,English_title,Original_title,Director,Production_country
0,1946-01-01,The Southerner,The Southerner,Jean Renoir,United States
1,1947-01-01,The Strike,Siréna,Karel Steklý,Czechoslovakia
2,1948-01-01,Hamlet,Hamlet,Laurence Olivier,United Kingdom
3,1949-01-01,Manon,Manon,Henri-Georges Clouzot,France
4,1950-01-01,Justice Is Done,Justice est faite,André Cayatte,France
...,...,...,...,...,...
65,2019-01-01,Joker,Joker,Todd Phillips,United States
66,2020-01-01,Nomadland,Nomadland,Chloé Zhao,United States
67,2021-01-01,Happening,L'Événement,Audrey Diwan,France
68,2022-01-01,All the Beauty and the Bloodshed,All the Beauty and the Bloodshed,Laura Poitras,United States


In [197]:
ven_larg_cl = ven_larg

#fusion des colonnes qui se sont retrouvées dispatchées à cause des noms des colonnes légérement différents 
#année 

ven_larg_cl['Yr'] = ven_larg_cl["('Year', '1990s')"].combine_first(ven_larg_cl['Year'])

#Nom 

ven_larg_cl['English_title'] = ven_larg_cl["('English Title', '1990s')"].combine_first(ven_larg_cl['English Title'])
ven_larg_cl['Original_title'] = ven_larg_cl["('Original Title', '1990s')"].combine_first(ven_larg_cl['Original Title'])

#réalisateur 
ven_larg_cl['Dir'] = ven_larg_cl["('Director(s)', '1990s')"].combine_first(ven_larg_cl['Director'])
ven_larg_cl['Dir'] = ven_larg_cl["Dir"].combine_first(ven_larg_cl['Director(s)'])
ven_larg_cl['Dir'] = ven_larg_cl["Dir"].combine_first(ven_larg_cl['Screenwriter'])
ven_larg_cl = ven_larg_cl.drop(["('Director(s)', '1990s')", 'Director','Director(s)', 'Screenwriter', "('Original Title', '1990s')", 'Original Title'
                               , 'Year', "('Year', '1990s')", "('English Title', '1990s')"], axis=1)
#sélection des colonnes 
ven_larg_cl = ven_larg_cl.iloc[::, 5:]

#renommer les colonnes qu'on vient de créer
ven_larg_cl.rename(columns={'Yr': 'Year', 'Dir':'Director'}, inplace=True)
ven_larg_cl = ven_larg_cl [~ven_larg_cl ['English_title'].isin(suppr)]
ven_larg_cl = ven_larg_cl.dropna(subset=['English_title'], how='all')

#conversion en type date 
ven_larg_cl = conv_date(ven_larg_cl)
ven_larg_cl = ven_larg_cl.sort_values(by='Year', ascending=True) #trier par date
ven_larg_cl = ven_larg_cl.reset_index(drop=True) #reset l'index du dataframe
ven_larg_cl

,Year,English_title,Original_title,Director
0,1953-01-01,Ugetsu,雨月物語,Kenji Mizoguchi
1,1953-01-01,Sadko,Садко,Aleksandr Ptushko
2,1953-01-01,Thérèse Raquin,Thérèse Raquin,Marcel Carné
3,1953-01-01,I Vitelloni,I Vitelloni,Federico Fellini
4,1953-01-01,Moulin Rouge,Moulin Rouge,John Huston
...,...,...,...,...
61,2019-01-01,About Endlessness,Om det oändliga,Roy Andersson
62,2020-01-01,Wife of a Spy,スパイの妻,Kiyoshi Kurosawa
63,2021-01-01,The Power of the Dog,The Power of the Dog,Jane Campion
64,2022-01-01,Bones and All,Bones and All,Luca Guadagnino


In [213]:
ven_gj_cl = ven_gj 
#renommer les colonnes
ven_gj_cl = renom_cannes(ven_gj_cl)

#garder les colonnes d'intérêt
ven_gj_cl = ven_gj_cl.iloc[::, :5]

#supprimer les lignes indésirables
ven_gj_cl = ven_gj_cl [~ven_gj_cl ['English_title'].isin(suppr)]

#supprimer les lignes aux valeurs manquantes 
ven_gj_cl  = ven_gj_cl.dropna(subset=['English_title'], how='all')

#supprimer les indices dans certains noms de films et remplacer avec le nom tout seul
ven_gj_cl['English_title'] = ven_gj_cl['English_title'].str.split('[', n=1).str[0]
ven_gj_cl['Original_title'] = ven_gj_cl['Original_title'].str.split('[', n=1).str[0]

ven_gj_cl = conv_date(ven_gj_cl )
ven_gj_cl = ven_gj_cl.reset_index(drop=True) #reset l'index du dataframe
ven_gj_cl 

,Year,English_title,Original_title,Director,Production_country
0,1951-01-01,A Streetcar Named Desire,A Streetcar Named Desire,Elia Kazan,United States
1,1952-01-01,The Curious Adventures of Mr. Wonderbird,La Bergère et le Ramoneur,Paul Grimault,France
2,1952-01-01,Mandy,Mandy,Alexander Mackendrick,United Kingdom
3,1954-01-01,Executive Suite *,Executive Suite *,Robert Wise,United States
4,1958-01-01,The Lovers,Les amants,Louis Malle,France
...,...,...,...,...,...
69,2019-01-01,An Officer and a Spy,J'accuse,Roman Polanski,"France, Italy"
70,2020-01-01,New Order,Nuevo Orden,Michel Franco,"Mexico, France"
71,2021-01-01,The Hand of God,È stata la mano di Dio,Paolo Sorrentino,Italy
72,2022-01-01,Saint Omer,Saint Omer,Alice Diop,France
